In [1]:
import numpy as np
import readline
%pylab inline
%load_ext autoreload
%autoreload 2
%precision 3

Populating the interactive namespace from numpy and matplotlib


'%.3f'

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
lemma = nltk.wordnet.WordNetLemmatizer()

tt = TweetTokenizer(preserve_case=False)

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
stops.update({"#semst", "...", "rt"})
ps = nltk.stem.porter.PorterStemmer()

def tokenize(t):
    tokens = tt.tokenize(t)
    tokens = [tok for tok in tokens if not ((len(tok) == 1 and not tok.isalpha()) or tok in stops)]
    tokens = [ (ps.stem(tok) if tok[0] != "#" else tok) for tok in tokens]
    return tokens

In [3]:
train = pd.read_csv("../semeval2016.6/StanceDataset/train.csv", encoding= 'unicode_escape', engine='python')
del train["Opinion Towards"]
del train["Sentiment"]
train["tokens"] = [ tokenize(t) for t in train["Tweet"].values ]
train["num_tokens"] = [len(tt) for tt in train["tokens"].values ]
train.head()

,Tweet,Target,Stance,tokens,num_tokens
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,"[@tedcruz, #handovertheserver, wipe, clean, 30...",14
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,"[hillari, best, choic, truli, want, continu, p...",9
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,"[@theview, think, countri, readi, femal, pre, ...",9
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,"[gave, unhealthi, amount, hard-earn, money, aw...",11
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,"[@portiaaboulg, thank, ad, list]",4


In [4]:
test = pd.read_csv("../semeval2016.6/StanceDataset/test.csv", encoding= 'unicode_escape', engine='python')
del test["Opinion Towards"]
del test["Sentiment"]
test["tokens"] = [ tokenize(t) for t in test["Tweet"].values ]
test["num_tokens"] = [len(tt) for tt in test["tokens"].values ]
test.head()

,Tweet,Target,Stance,tokens,num_tokens
0,He who exalts himself shall be humbled; a...,Atheism,AGAINST,"[exalt, shall, humbl, humbl, shall, exalted.ma...",7
1,RT @prayerbullets: I remove Nehushtan -previou...,Atheism,AGAINST,"[@prayerbullet, remov, nehushtan, previou, mov...",12
2,@Brainman365 @heidtjj @BenjaminLives I have so...,Atheism,AGAINST,"[@brainman365, @heidtjj, @benjaminl, sought, t...",11
3,#God is utterly powerless without Human interv...,Atheism,AGAINST,"[#god, utterli, powerless, without, human, int...",6
4,@David_Cameron Miracles of #Multiculturalism...,Atheism,AGAINST,"[@david_cameron, miracl, #multiculturalism, mi...",13


In [5]:
from submodular.models2 import *
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer

In [64]:
lst = ['AGAINST', 'FAVOR']
train_target = train.query('Target == "Climate Change is a Real Concern" & Stance == @lst').reset_index()
test_target = test.query('Target == "Climate Change is a Real Concern" & Stance == @lst').reset_index()
del train_target['index']
del test_target['index']
y_tr = np.ones(len(train_target), dtype = np.int8)
y_te = np.ones(len(test_target), dtype = np.int8)
y_tr[np.where(train_target.Stance.values == "AGAINST")] = -1
y_te[np.where(test_target.Stance.values == "AGAINST")] = -1
train_target["y"] = y_tr
test_target["y"] = y_te
train_target.head()

,Tweet,Target,Stance,tokens,num_tokens,y
0,"We cant deny it, its really happening. #SemST",Climate Change is a Real Concern,FAVOR,"[cant, deni, realli, happen]",4,1
1,RT @cderworiz: Timelines are short. Strategy m...,Climate Change is a Real Concern,FAVOR,"[@cderworiz, timelin, short, strategi, must, p...",12,1
2,SO EXCITING! Meaningful climate change action ...,Climate Change is a Real Concern,FAVOR,"[excit, meaning, climat, chang, action, way, #...",8,1
3,"Delivering good jobs for Albertans, maintainin...",Climate Change is a Real Concern,FAVOR,"[deliv, good, job, albertan, maintain, stabl, ...",15,1
4,@davidswann says he wants carbon fund to be sp...,Climate Change is a Real Concern,FAVOR,"[@davidswann, say, want, carbon, fund, spent, ...",12,1


In [65]:
steps = ["tok", "idf-bow", "exp", "greedy-diff"]
model = get_model(steps)
model

([('tok', PandasSelector(tokens)),
  ('idf-bow', BoW(tf=counts, weights=idf)),
  ('exp', ExpK()),
  ('greedy-diff', MMDGreedy(C=3.0,lambda=0.21,m=4))],
 [('gamma', 'exp'), ('C', 'greedy-diff'), ('lambdaa', 'greedy-diff')])

In [66]:
# ps = PandasSelector('tokens')
# y = train_target.y.values
# X = ps.fit_transform(train_target, y)

# idf = BoW(weights='idf')
# Xn = idf.fit_transform(X, y)
# k = ExpK(gamma = 2.0)
# K = k.fit_transform(Xn, y)
# mmd = MMDGreedy(mk, 1.0, 0.2)
# mmd.fit(K, y)

# print(mmd.score(K, y))
# train_target.iloc[mmd.idxs]
pipe, _hyperparams = get_model(steps, mk = 5)
pp = Pipeline(pipe)
pp.fit(train_target, train_target.y.values)
print(pp.score(train_target, train_target.y.values), pp.score(test_target, test_target.y.values))
train_target.iloc[pp.steps[-1][1].idxs]

Jul-02 22:48:05 INFO [MMDGreedy:275]=> fitted MMDGreedy(C=3.0,lambda=0.21,m=5)


0.7534591194968554 0.5842572062084257


,Tweet,Target,Stance,tokens,num_tokens,y
193,GlblWarmingNews And the tooth fairy might be c...,Climate Change is a Real Concern,AGAINST,"[glblwarmingnew, tooth, fairi, might, caus, ki...",10,-1
44,Climate change?! Explain the definition of cli...,Climate Change is a Real Concern,AGAINST,"[climat, chang, explain, definit, climat, chan...",11,-1
192,Global Warming! Global Warming! Global Warming...,Climate Change is a Real Concern,AGAINST,"[global, warm, global, warm, global, warm, oh,...",9,-1
169,"Noted climatologist, marine biologist,& oceano...",Climate Change is a Real Concern,AGAINST,"[note, climatologist, marin, biologist, oceano...",17,-1
199,If ther clmate change alarmists come out in se...,Climate Change is a Real Concern,AGAINST,"[ther, clmate, chang, alarmist, come, sept, ye...",14,-1
152,#Mission:#Climate @ home > Simplify (by @BeZer...,Climate Change is a Real Concern,FAVOR,"[#mission, #climate, home, simplifi, @bezerowa...",7,1
70,A changing planet: 4 places where climate chan...,Climate Change is a Real Concern,FAVOR,"[chang, planet, place, climat, chang, take, toll]",7,1
84,The last thing #Australia or the world needs a...,Climate Change is a Real Concern,FAVOR,"[last, thing, #australia, world, need, vast, e...",15,1
224,Vallaud-Belkacem: Are we the generation who kn...,Climate Change is a Real Concern,FAVOR,"[vallaud-belkacem, gener, knew, would, happen,...",9,1
137,"Feeling like #SCOTUS got it right, right, righ...",Climate Change is a Real Concern,FAVOR,"[feel, like, #scotus, got, right, right, right...",10,1


In [67]:
from commons.functions import bacc

HYPERPARAMS_GRID = {
    "lambdaa": [0.1, 0.2],
    "C": [0.1, 1.0],
    "gamma": [1.0, 2.0, 4.0]
}

mk = 5
CV = 3
N_JOBS = 6

scorer = make_scorer(bacc)

pipe, _hyperparams = get_model(steps, mk = mk)
name = "_".join(steps)
hyperparams = {}
for hp in _hyperparams:
    hp_name, step = hp
    hyperparams["{}__{}".format(step, hp_name)] = HYPERPARAMS_GRID[hp_name]
    if step == "greedy-div" and hp_name == "lambdaa":
        hyperparams["{}__{}".format(step, hp_name)] = HYPERPARAMS_GRID["lambda2"]
print("{}, m={}, params={}".format(name, mk, ";".join(hyperparams.keys()) ))

clf = GridSearchCV(Pipeline(pipe), hyperparams, 
            cv = CV, n_jobs = N_JOBS, pre_dispatch = N_JOBS, verbose = 0, scoring = scorer, 
            return_train_score = False, error_score = 0.5)

clf.fit(train_target, train_target.y.values)
print(clf, clf.score(train_target, train_target.y.values), clf.score(test_target, test_target.y.values))
train_target.iloc[clf.best_estimator_.steps[-1][1].idxs]

tok_idf-bow_exp_greedy-diff, m=5, params=exp__gamma;greedy-diff__C;greedy-diff__lambdaa


Jul-02 22:48:20 INFO [MMDGreedy:275]=> fitted MMDGreedy(C=0.1,lambda=0.2,m=5)


GridSearchCV(cv=3, error_score=0.5,
             estimator=Pipeline(steps=[('tok', PandasSelector(tokens)),
                                       ('idf-bow', BoW(weights='idf')),
                                       ('exp', ExpK()),
                                       ('greedy-diff', MMDGreedy(mk=5))]),
             n_jobs=6,
             param_grid={'exp__gamma': [1.0, 2.0, 4.0],
                         'greedy-diff__C': [0.1, 1.0],
                         'greedy-diff__lambdaa': [0.1, 0.2]},
             pre_dispatch=6, scoring=make_scorer(bacc)) 0.7122641509433962 0.6611234294161124


,Tweet,Target,Stance,tokens,num_tokens,y
193,GlblWarmingNews And the tooth fairy might be c...,Climate Change is a Real Concern,AGAINST,"[glblwarmingnew, tooth, fairi, might, caus, ki...",10,-1
44,Climate change?! Explain the definition of cli...,Climate Change is a Real Concern,AGAINST,"[climat, chang, explain, definit, climat, chan...",11,-1
192,Global Warming! Global Warming! Global Warming...,Climate Change is a Real Concern,AGAINST,"[global, warm, global, warm, global, warm, oh,...",9,-1
169,"Noted climatologist, marine biologist,& oceano...",Climate Change is a Real Concern,AGAINST,"[note, climatologist, marin, biologist, oceano...",17,-1
199,If ther clmate change alarmists come out in se...,Climate Change is a Real Concern,AGAINST,"[ther, clmate, chang, alarmist, come, sept, ye...",14,-1
152,#Mission:#Climate @ home > Simplify (by @BeZer...,Climate Change is a Real Concern,FAVOR,"[#mission, #climate, home, simplifi, @bezerowa...",7,1
70,A changing planet: 4 places where climate chan...,Climate Change is a Real Concern,FAVOR,"[chang, planet, place, climat, chang, take, toll]",7,1
84,The last thing #Australia or the world needs a...,Climate Change is a Real Concern,FAVOR,"[last, thing, #australia, world, need, vast, e...",15,1
224,Vallaud-Belkacem: Are we the generation who kn...,Climate Change is a Real Concern,FAVOR,"[vallaud-belkacem, gener, knew, would, happen,...",9,1
137,"Feeling like #SCOTUS got it right, right, righ...",Climate Change is a Real Concern,FAVOR,"[feel, like, #scotus, got, right, right, right...",10,1


In [68]:
steps2 = ["tok", "idf-bow", "length"]
model2 = get_model(steps2)

HYPERPARAMS_GRID = {
    "lambdaa": [0.1, 0.2],
    "C": [0.1, 1.0],
    "gamma": [1.0, 2.0, 4.0]
}

pipe2, _hyperparams2 = get_model(steps2, mk = mk)
name2 = "_".join(steps2)
hyperparams2 = {}
for hp in _hyperparams2:
    hp_name, step = hp
    hyperparams2["{}__{}".format(step, hp_name)] = HYPERPARAMS_GRID[hp_name]

clf = GridSearchCV(Pipeline(pipe2), hyperparams2, 
            cv = CV, n_jobs = N_JOBS, pre_dispatch = N_JOBS, verbose = 0, scoring = scorer, 
            return_train_score = False, error_score = 0.5)

clf.fit(train_target, train_target.y.values)
print(clf, clf.score(train_target, train_target.y.values), clf.score(test_target, test_target.y.values))
train_target.iloc[clf.best_estimator_.steps[-1][1].idxs]

Jul-02 22:48:25 INFO [LengthComp:275]=> fitted LengthComp(C=0.1,gamma=1.0,m=5)


GridSearchCV(cv=3, error_score=0.5,
             estimator=Pipeline(steps=[('tok', PandasSelector(tokens)),
                                       ('idf-bow', BoW(weights='idf')),
                                       ('length', LengthComp(mk=5))]),
             n_jobs=6,
             param_grid={'length__C': [0.1, 1.0],
                         'length__gamma': [1.0, 2.0, 4.0]},
             pre_dispatch=6, scoring=make_scorer(bacc)) 0.5117924528301887 0.5


,Tweet,Target,Stance,tokens,num_tokens,y
166,[Man] cannot change the climate of any country...,Climate Change is a Real Concern,AGAINST,"[man, cannot, chang, climat, countri, add, new...",12,-1
165,I'm sick of celebrities who think being a well...,Climate Change is a Real Concern,AGAINST,"[i'm, sick, celebr, think, well, known, actor,...",13,-1
167,RedScareBot: Simple Malarkey RT gingerdr Globa...,Climate Change is a Real Concern,AGAINST,"[redscarebot, simpl, malarkey, gingerdr, globa...",13,-1
199,If ther clmate change alarmists come out in se...,Climate Change is a Real Concern,AGAINST,"[ther, clmate, chang, alarmist, come, sept, ye...",14,-1
169,"Noted climatologist, marine biologist,& oceano...",Climate Change is a Real Concern,AGAINST,"[note, climatologist, marin, biologist, oceano...",17,-1
144,@lancet #climatehealth report says we need am...,Climate Change is a Real Concern,FAVOR,"[@lancet, #climatehealth, report, say, need, a...",15,1
219,until newyork london frankfurt knee deep in se...,Climate Change is a Real Concern,FAVOR,"[newyork, london, frankfurt, knee, deep, sea, ...",15,1
18,Go Dutch New meaning: citizens take their gove...,Climate Change is a Real Concern,FAVOR,"[go, dutch, new, mean, citizen, take, govern, ...",15,1
65,Did u know th #SCOTUS is 2 Decide If #Coal Pla...,Climate Change is a Real Concern,FAVOR,"[u, know, th, #scotus, decid, #coal, plant, ke...",16,1
84,The last thing #Australia or the world needs a...,Climate Change is a Real Concern,FAVOR,"[last, thing, #australia, world, need, vast, e...",15,1


In [69]:
steps2 = ["tok", "idf-bow", "cos", "random"]
model2 = get_model(steps2)

HYPERPARAMS_GRID = {
    "lambdaa": [0.1, 0.2],
    "C": [0.1, 1.0],
    "gamma": [1.0, 2.0, 4.0]
}

pipe2, _hyperparams2 = get_model(steps2, mk = mk)
name2 = "_".join(steps2)
hyperparams2 = {}
for hp in _hyperparams2:
    hp_name, step = hp
    hyperparams2["{}__{}".format(step, hp_name)] = HYPERPARAMS_GRID[hp_name]

clf = GridSearchCV(Pipeline(pipe2), hyperparams2, 
            cv = CV, n_jobs = N_JOBS, pre_dispatch = N_JOBS, verbose = 0, scoring = scorer, 
            return_train_score = False, error_score = 0.5)

clf.fit(train_target, train_target.y.values)
print(clf, clf.score(train_target, train_target.y.values), clf.score(test_target, test_target.y.values))
train_target.iloc[clf.best_estimator_.steps[-1][1].idxs]

Jul-02 22:48:27 INFO [RandomComp:275]=> fitted RandomComp(C=0.1,m=5)


GridSearchCV(cv=3, error_score=0.5,
             estimator=Pipeline(steps=[('tok', PandasSelector(tokens)),
                                       ('idf-bow', BoW(weights='idf')),
                                       ('cos', CosineK()),
                                       ('random', RandomComp(mk=5))]),
             n_jobs=6, param_grid={'random__C': [0.1, 1.0]}, pre_dispatch=6,
             scoring=make_scorer(bacc)) 0.5185534591194969 0.5480413895048041


,Tweet,Target,Stance,tokens,num_tokens,y
9,The Climate Change people are disgusting assho...,Climate Change is a Real Concern,AGAINST,"[climat, chang, peopl, disgust, asshol, money,...",11,-1
31,"BREAKING: This just in, an ocean wave just bro...",Climate Change is a Real Concern,AGAINST,"[break, ocean, wave, broke, inch, beach, normal]",7,-1
44,Climate change?! Explain the definition of cli...,Climate Change is a Real Concern,AGAINST,"[climat, chang, explain, definit, climat, chan...",11,-1
47,"Last time I checked, Al Gore is a politician, ...",Climate Change is a Real Concern,AGAINST,"[last, time, check, al, gore, politician, scie...",9,-1
165,I'm sick of celebrities who think being a well...,Climate Change is a Real Concern,AGAINST,"[i'm, sick, celebr, think, well, known, actor,...",13,-1
0,"We cant deny it, its really happening. #SemST",Climate Change is a Real Concern,FAVOR,"[cant, deni, realli, happen]",4,1
1,RT @cderworiz: Timelines are short. Strategy m...,Climate Change is a Real Concern,FAVOR,"[@cderworiz, timelin, short, strategi, must, p...",12,1
2,SO EXCITING! Meaningful climate change action ...,Climate Change is a Real Concern,FAVOR,"[excit, meaning, climat, chang, action, way, #...",8,1
3,"Delivering good jobs for Albertans, maintainin...",Climate Change is a Real Concern,FAVOR,"[deliv, good, job, albertan, maintain, stabl, ...",15,1
4,@davidswann says he wants carbon fund to be sp...,Climate Change is a Real Concern,FAVOR,"[@davidswann, say, want, carbon, fund, spent, ...",12,1


In [70]:
steps2 = ["tok", "idf-bow", "cos", "nn"]
model2 = get_model(steps2)

HYPERPARAMS_GRID = {
    "lambdaa": [0.1, 0.2],
    "C": [0.1, 1.0],
    "gamma": [1.0, 2.0, 4.0]
}

pipe2, _hyperparams2 = get_model(steps2, mk = mk)
name2 = "_".join(steps2)
hyperparams2 = {}
for hp in _hyperparams2:
    hp_name, step = hp
    hyperparams2["{}__{}".format(step, hp_name)] = HYPERPARAMS_GRID[hp_name]

clf = GridSearchCV(Pipeline(pipe2), hyperparams2, 
            cv = CV, n_jobs = N_JOBS, pre_dispatch = N_JOBS, verbose = 0, scoring = scorer, 
            return_train_score = False, error_score = 0.5)

clf.fit(train_target, train_target.y.values)
print(clf, clf.score(train_target, train_target.y.values), clf.score(test_target, test_target.y.values))
train_target.iloc[clf.best_estimator_.steps[-1][1].idxs]

Jul-02 22:48:31 INFO [NNComp:275]=> fitted NNComp(C=0.1,m=5)


GridSearchCV(cv=3, error_score=0.5,
             estimator=Pipeline(steps=[('tok', PandasSelector(tokens)),
                                       ('idf-bow', BoW(weights='idf')),
                                       ('cos', CosineK()),
                                       ('nn', NNComp(mk=5))]),
             n_jobs=6, param_grid={'nn__C': [0.1, 1.0]}, pre_dispatch=6,
             scoring=make_scorer(bacc)) 0.8393081761006289 0.5801921655580192


,Tweet,Target,Stance,tokens,num_tokens,y
193,GlblWarmingNews And the tooth fairy might be c...,Climate Change is a Real Concern,AGAINST,"[glblwarmingnew, tooth, fairi, might, caus, ki...",10,-1
44,Climate change?! Explain the definition of cli...,Climate Change is a Real Concern,AGAINST,"[climat, chang, explain, definit, climat, chan...",11,-1
167,RedScareBot: Simple Malarkey RT gingerdr Globa...,Climate Change is a Real Concern,AGAINST,"[redscarebot, simpl, malarkey, gingerdr, globa...",13,-1
31,"BREAKING: This just in, an ocean wave just bro...",Climate Change is a Real Concern,AGAINST,"[break, ocean, wave, broke, inch, beach, normal]",7,-1
165,I'm sick of celebrities who think being a well...,Climate Change is a Real Concern,AGAINST,"[i'm, sick, celebr, think, well, known, actor,...",13,-1
70,A changing planet: 4 places where climate chan...,Climate Change is a Real Concern,FAVOR,"[chang, planet, place, climat, chang, take, toll]",7,1
152,#Mission:#Climate @ home > Simplify (by @BeZer...,Climate Change is a Real Concern,FAVOR,"[#mission, #climate, home, simplifi, @bezerowa...",7,1
84,The last thing #Australia or the world needs a...,Climate Change is a Real Concern,FAVOR,"[last, thing, #australia, world, need, vast, e...",15,1
224,Vallaud-Belkacem: Are we the generation who kn...,Climate Change is a Real Concern,FAVOR,"[vallaud-belkacem, gener, knew, would, happen,...",9,1
140,If you do not repent your emission of CO2 you ...,Climate Change is a Real Concern,FAVOR,"[repent, emiss, co2, end, hell, #bible]",6,1
